# Training an Iris classifier using XGBoost

In this notebook, we'll show how to train a classifier trained on the [iris data set](https://archive.ics.uci.edu/ml/datasets/iris) using XGBoost.

## Install Dependencies
First, we'll install our dependencies:

In [0]:
pip install xgboost==0.90 scikit-learn==0.21.* onnxmltools==1.5.* boto3==1.*

## Load the data
We can use scikit-learn to load the Iris dataset:

In [0]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

## Train the model
We'll use XGBoost's [`XGBClassifier`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier) to train the model:

In [0]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier()
xgb_model = xgb_model.fit(X_train, y_train)

print("Test data accuracy of the xgb classifier is {:.2f}".format(xgb_model.score(X_test, y_test)))  # Accuracy should be > 90%

## Export the model
Now we can export the model in the ONNX format:

In [0]:
from onnxmltools.convert import convert_xgboost
from onnxconverter_common.data_types import FloatTensorType

onnx_model = convert_xgboost(xgb_model, initial_types=[("input", FloatTensorType([1, 4]))])

with open("gbtree.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## Upload the model to AWS

Cortex loads models from AWS, so we need to upload the exported model.

Set these variables to configure your AWS credentials and model upload path:

In [0]:
AWS_ACCESS_KEY_ID = "" #@param {type:"string"}
AWS_SECRET_ACCESS_KEY = "" #@param {type:"string"}
S3_UPLOAD_PATH = "s3://my-bucket/iris-classifier/gbtree.onnx" #@param {type:"string"}

import sys
import re

if AWS_ACCESS_KEY_ID == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_ACCESS_KEY_ID"), file=sys.stderr)

elif AWS_SECRET_ACCESS_KEY == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_SECRET_ACCESS_KEY"), file=sys.stderr)

else:
    try:
        bucket, key = re.match("s3://(.+?)/(.+)", S3_UPLOAD_PATH).groups()
    except:
        print("\033[91m{}\033[00m".format("ERROR: Invalid s3 path (should be of the form s3://my-bucket/path/to/file)"), file=sys.stderr)

Upload the model to S3:

In [0]:
import boto3

s3 = boto3.client("s3", aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
print("Uploading {} ...".format(S3_UPLOAD_PATH), end = '')
s3.upload_file("gbtree.onnx", bucket, key)
print(" ✓")